In [1]:
import json
from pathlib import Path

import pandas as pd

In [2]:
def load_interactions(jsonl_path: Path) -> pd.DataFrame:
    rows = []
    with jsonl_path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)

            user = obj.get("user_id")
            parent_asin = obj.get("PARENT_ASIN") or obj.get("parent_asin")
            rating = obj.get("rating")

            # 필수 필드 없으면 스킵
            if user is None or parent_asin is None or rating is None:
                continue

            rows.append(
                {
                    "user_id": user,
                    "item_id": parent_asin,
                    "rating": float(rating),
                }
            )

    df = pd.DataFrame(rows)
    return df

In [3]:
jsonl_path = Path("../data/Amazon_Fashion_v1.jsonl")
df = load_interactions(jsonl_path)
num_rows = len(df)
num_users = df["user_id"].nunique()
num_items = df["item_id"].nunique()

In [4]:
pd.set_option("display.float_format", lambda x: f"{x:.2f}")

In [6]:
print("=== 기본 규모 EDA ===")
print(f"- 평가 데이터 수 (rows): {num_rows:,}")
print(f"- 유저 수 (unique user_id): {num_users:,}")
print(f"- 상품 수 (unique PARENT_ASIN): {num_items:,}")

# 참고로 한 번 더: 유저당 평균 평가 수 / 아이템당 평균 평가 수
print("\n=== 밀도(희소성) 간단 확인 ===")
print(f"- 유저당 평균 평가 수: {num_rows / num_users:.2f}")
print(f"- 아이템당 평균 평가 수: {num_rows / num_items:.2f}")

# 유저당 평가 개수 분포 대략 보기
user_counts = df["user_id"].value_counts()
print("\n=== 유저당 평가 수 분포 예시 ===")
print(user_counts.describe())

# 아이템당 평가 개수 분포 대략 보기
item_counts = df["item_id"].value_counts()
print("\n=== 아이템당 평가 수 분포 예시 ===")
print(item_counts.describe())

=== 기본 규모 EDA ===
- 평가 데이터 수 (rows): 1,594,222
- 유저 수 (unique user_id): 1,382,557
- 상품 수 (unique PARENT_ASIN): 462,935

=== 밀도(희소성) 간단 확인 ===
- 유저당 평균 평가 수: 1.15
- 아이템당 평균 평가 수: 3.44

=== 유저당 평가 수 분포 예시 ===
count   1382557.00
mean          1.15
std           0.67
min           1.00
25%           1.00
50%           1.00
75%           1.00
max         179.00
Name: count, dtype: float64

=== 아이템당 평가 수 분포 예시 ===
count   462935.00
mean         3.44
std         24.47
min          1.00
25%          1.00
50%          1.00
75%          3.00
max       7554.00
Name: count, dtype: float64
